In [1]:
%pip install psycopg2

  Using cached psycopg2-2.9.9.tar.gz (384 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp312-cp312-macosx_11_0_arm64.whl size=133173 sha256=5ed7f12b993103355a785a5526660e78869dfd28387368a439ce3d87a81fe6e9
  Stored in directory: /Users/haolu/Library/Caches/pip/wheels/ff/ac/80/7ccec163e3d05ae2112311b895132409b9abfd7e0c1c6b5183
Successfully built psycopg2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import psycopg2 as pg
from datetime import datetime, timezone, timedelta
import time
from binance.spot import Spot
from sql import *


class Collector:
    def __init__(self):
        self.storage = pg.connect(user="admin",
                        password="quest",
                        host="127.0.0.1",
                        port="8812",
                        database="qdb")
        self.retriever = Spot()
        
        # init time series db
        cursor = self.storage.cursor()

        cursor.execute(SQL_CREATE_CANDLE)
        self.storage.commit()

        print('self')


    def retrieve(self, start, end):
        pass

    def auto_retrieve(self, start=None):
        if start is None:
            
            pass
    

    def most_recent(self):
        cursor = self.storage.cursor()
        cursor.execute("")


In [ ]:
import random
import psycopg2 as pg
import datetime as dt
import time

connection = pg.connect(user="admin",
                        password="quest",
                        host="127.0.0.1",
                        port="8812",
                        database="qdb")
cursor = connection.cursor()

# Create table stock_prices
cursor.execute("CREATE TABLE IF NOT EXISTS stock_prices (stock symbol, stockPrice double, createdDatetime timestamp) timestamp(createdDatetime)")

try:
  print("Inserting rows into table 'stock_prices' - press Ctrl-C to stop")
  while True:
    # For testing purposes, create random stock prices for Tesla
    symbol = "TSLA"
    price = round(random.uniform(760.00, 810.00), 2)
    now = dt.datetime.utcnow()

    print("Inserting into 'stock_prices': %s %s %s" % (symbol, price, now))

    cursor.execute("""
      INSERT INTO stock_prices
      VALUES (%s, %s, %s);
      """, (symbol, price, now))
    connection.commit()
    time.sleep(0.5)

except KeyboardInterrupt:
    get_row_count = cursor.execute("SELECT count() FROM stock_prices")
    rowcount = cursor.fetchall()
    print("\nFinished, table has %s total rows" % (rowcount[0]))

    pass

finally:
  if (connection):
    cursor.close()
    connection.close()
    print("QuestDB connection closed")

In [ ]:
import psycopg2 as pg
import time
import os
import datetime as dt
from iexfinance.stocks import Stock

# Use dotenv to load secrets
from dotenv import load_dotenv
load_dotenv()

realtime_price = None
symbol = ''
now = None

# DB connection
connection = pg.connect(user="admin",
                        password="quest",
                        host="127.0.0.1",
                        port="8812",
                        database="qdb")

cursor = connection.cursor()

# Store API key in `.env` next to this file in the format API_TOKEN=ac8cb618...
my_token = os.environ.get("API_TOKEN")

def get_stock_by_symbol(stock_symbol):
  df = Stock(stock_symbol, token=my_token, output_format = 'pandas').get_quote()
  realtime_price = df['iexRealtimePrice'][0]
  symbol = df['symbol'][0]
  now = dt.datetime.utcnow()

try:
  print("Inserting rows into table 'stock_prices' - press Ctrl-C to stop")
  # Create table stock_prices
  cursor.execute("CREATE TABLE IF NOT EXISTS stock_prices (stock symbol, stockPrice double, createdDatetime timestamp) timestamp(createdDatetime)")

  get_stock_by_symbol("TSLA")

  while realtime_price:
      get_stock_by_symbol("TSLA")
      print("Inserting into 'stock_prices': %s %s %s" % (symbol, price, now))

      cursor.execute("""
        INSERT INTO stock_prices
        VALUES (%s, %s, %s);
        """, (symbol, price, now))
      connection.commit()
      time.sleep(0.5)
  else:
    print("\nWarning: market is currently closed, exiting.")
    print("Consider using dummy data instead to conserve API quotas\n")

except KeyboardInterrupt:
    get_row_count = cursor.execute("SELECT count() FROM stock_prices")
    rowcount = cursor.fetchall()
    print("\nFinished, table has %s total rows" % (rowcount[0]))

    pass

finally:
  if (connection):
    cursor.close()
    connection.close()
    print("QuestDB connection closed")